In [3]:
import pandas as pd

%load_ext autoreload
%autoreload 2

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper'.format(folder)
bed_folder = '{}/bed'.format(folder)


splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

alt_exons_bedfile = '{}/exons.bed'.format(alternative_feature_folder)
constitutive_bedfile = '{}/exons.bed'.format(constitutive_feature_folder)

bedfiles = alt_exons_bedfile, constitutive_bedfile

# splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(csv_folder), index_col=0)
# print(splicing_feature_data.shape)
# splicing_feature_data.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pybedtools
constitutive_bed = pybedtools.BedTool(constitutive_bedfile)
names = [x.name for x in constitutive_bed]

constitutive_feature_data = pd.DataFrame(index=names)

In [ ]:
len(constitutive_bed)

In [ ]:
hg19_fasta = '/projects/ps-yeolab/genomes/hg19/gencode/v19/GRCh37.p13.genome.fa'

## Splice site strength

In [ ]:
from poshsplice.splicestrength import score_exons


constitutive_scores = score_exons(constitutive_bedfile, 'hg19', hg19_fasta)
constitutive_scores.head(10)

In [ ]:
constitutive_scores.shape

In [ ]:
ls $csv_folder/constitutive*

### Write constitutive data

In [ ]:
constitutive_scores.to_csv('{}/maxentscan.csv'.format(constitutive_feature_folder))

## Alternative scores

In [ ]:
from poshsplice.splicestrength import score_exons

alt_exons_scores = score_exons(alt_exons_bedfile, 'hg19', hg19_fasta)
alt_exons_scores.head()

In [ ]:
! wc -l $alt_exons_bedfile

In [ ]:
alt_exons_scores.shape

In [ ]:
alt_exons_scores.to_csv('{}/maxentscan.csv'.format(alternative_feature_folder))

##  Compare alt and constitutve

In [ ]:
alt_exons_scores['exon_type'] = 'alternative'
constitutive_scores['exon_type'] = 'constitutive'


splice_site_scores = pd.concat([alt_exons_scores, constitutive_scores])
splice_site_scores.head()

In [ ]:
splice_site_scores = splice_site_scores.rename(columns={'index': 'event_name'})
splice_site_scores.head()

In [ ]:
three_prime = splice_site_scores[['splice_site_3p_score', 'splice_site_3p_seq', 'exon_type']]
three_prime = three_prime.rename(columns={'splice_site_3p_score': 'score', 'splice_site_3p_seq': 'seq'})
three_prime['splice_site'] = "3'"
three_prime.head()

In [ ]:
five_prime = splice_site_scores[['splice_site_5p_score', 'splice_site_5p_seq', 'exon_type']]
five_prime = five_prime.rename(columns={'splice_site_5p_score': 'score', 'splice_site_5p_seq': 'seq'})
five_prime['splice_site'] = "5'"
five_prime.head()

In [ ]:
splice_site_scores_tidy = pd.concat([five_prime.reset_index(), three_prime.reset_index()], ignore_index=True)
splice_site_scores_tidy.head()

In [ ]:
splice_site_scores

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='ticks', context='notebook')

%matplotlib inline

fig, ax = plt.subplots(figsize=(2, 2))
sns.boxplot(hue='exon_type', y='score', data=splice_site_scores_tidy, x='splice_site', 
            linewidth=1, fliersize=2, order=["3'", "5'"])
sns.despine()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='ticks', context='notebook')

%matplotlib inline

fig, ax = plt.subplots(figsize=(2, 2))
sns.violinplot(hue='exon_type', y='score', data=splice_site_scores_tidy, x='splice_site', 
            linewidth=1, order=["3'", "5'"])
sns.despine()